In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://ucsbgauchos.com/sports/mens-basketball/stats'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [4]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [5]:
#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [6]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data

['30', 'Swenson, Stephan', '33', '33', '982', '29.8', '149', '339', '.440', '89', '206', '.432', '54', '68', '.794', '441', '13.4', '15', '106', '121', '3.7', '77', '144', '72', '59', '6', 'View Bio']
['11', 'Pohto, Kenny', '34', '34', '871', '25.6', '156', '251', '.622', '0', '1', '.000', '88', '131', '.672', '400', '11.8', '83', '127', '210', '6.2', '64', '54', '67', '18', '12', 'View Bio']
['10', '10 Fontenet II, Jason', '34', '34', '1000', '29.4', '109', '262', '.416', '34', '107', '.318', '84', '109', '.771', '336', '9.9', '36', '167', '203', '6.0', '89', '91', '56', '29', '10', 'View Bio']
['02', 'Turner, Deuce', '34', '3', '664', '19.5', '113', '273', '.414', '43', '136', '.316', '57', '68', '.838', '326', '9.6', '3', '59', '62', '1.8', '39', '49', '44', '18', '2', 'View Bio']
['04', 'Anderson, Cole', '34', '34', '973', '28.6', '99', '214', '.463', '81', '171', '.474', '23', '25', '.920', '302', '8.9', '6', '60', '66', '1.9', '50', '35', '24', '23', '3', 'View Bio']
['07', 'Smit

In [7]:

#Export this into a csv
df.to_csv(r'\Users\ruyek\Downloads\ucsb_player_stats.csv', index=False, encoding = "utf-8")